In [1]:
from fetchYahooStockData import metadata_price_action, extended_stock_stats, basic_stats, current_sp500_symbols, parse_value, current_dow30_symbols, current_nasdaq100_symbols
import numpy as np
import pandas as pd
import math

In [5]:
symbol="TTWO"
timeInterval = "1d"
timeRange = "1y"

In [4]:
sp500symbols = np.array(current_sp500_symbols())
sp500symbols

array(['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP',
       'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN',
       'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN',
       'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP',
       'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS',
       'APA', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ANET', 'AJG', 'AIZ', 'T',
       'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC',
       'BK', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'BIIB', 'BLK', 'BA',
       'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF.B', 'CHRW',
       'COG', 'CDNS', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT',
       'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CERN', 'CF',
       'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS',
       'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH',
       'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'COO', 'CPRT',


In [5]:
dow30symbols = np.array(current_dow30_symbols())
dow30symbols

array(['MMM', 'AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO',
       'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD',
       'MRK', 'MSFT', 'NKE', 'PG', 'CRM', 'TRV', 'UNH', 'VZ', 'V', 'WBA',
       'WMT', 'DIS'], dtype='<U4')

In [6]:
nasdaq100symbols = np.array(current_nasdaq100_symbols())
nasdaq100symbols

array(['ATVI', 'ADBE', 'AMD', 'ALXN', 'ALGN', 'GOOGL', 'GOOG', 'AMZN',
       'AEP', 'AMGN', 'ADI', 'ANSS', 'AAPL', 'AMAT', 'ASML', 'TEAM',
       'ADSK', 'ADP', 'BIDU', 'BIIB', 'BKNG', 'AVGO', 'CDNS', 'CDW',
       'CERN', 'CHTR', 'CHKP', 'CTAS', 'CSCO', 'CTSH', 'CMCSA', 'CPRT',
       'COST', 'CSX', 'DXCM', 'DOCU', 'DLTR', 'EBAY', 'EA', 'EXC', 'FB',
       'FAST', 'FISV', 'FOXA', 'FOX', 'GILD', 'IDXX', 'ILMN', 'INCY',
       'INTC', 'INTU', 'ISRG', 'JD', 'KDP', 'KLAC', 'KHC', 'LRCX', 'LULU',
       'MAR', 'MRVL', 'MTCH', 'MXIM', 'MELI', 'MCHP', 'MU', 'MSFT',
       'MRNA', 'MDLZ', 'MNST', 'NTES', 'NFLX', 'NVDA', 'NXPI', 'ORLY',
       'OKTA', 'PCAR', 'PAYX', 'PYPL', 'PTON', 'PEP', 'PDD', 'QCOM',
       'REGN', 'ROST', 'SGEN', 'SIRI', 'SWKS', 'SPLK', 'SBUX', 'SNPS',
       'TMUS', 'TSLA', 'TXN', 'TCOM', 'VRSN', 'VRSK', 'VRTX', 'WBA',
       'WDAY', 'XEL', 'XLNX', 'ZM'], dtype='<U5')

In [23]:
allSymbols = np.concatenate((nasdaq100symbols,sp500symbols), axis=None)
setAllSymbols = np.array(list(set(allSymbols)))
setAllSymbols


array(['PFG', 'PNR', 'GLW', 'WDAY', 'PSX', 'AMD', 'IFF', 'PDD', 'BIDU',
       'PSA', 'WHR', 'ASML', 'ARE', 'EOG', 'VRTX', 'VNO', 'TWTR', 'NWS',
       'OKTA', 'KEYS', 'WRB', 'COO', 'GPC', 'CSCO', 'ROP', 'VNT', 'ROST',
       'AMAT', 'DISH', 'HSY', 'LUV', 'FLIR', 'COST', 'SPLK', 'FIS', 'TYL',
       'KIM', 'NTRS', 'SPGI', 'IRM', 'REGN', 'CMI', 'BIIB', 'BK', 'XRAY',
       'REG', 'ANET', 'CB', 'PLD', 'PNW', 'FCX', 'IPGP', 'MDT', 'CHRW',
       'LYB', 'NWSA', 'ITW', 'MTCH', 'PEAK', 'MAR', 'KEY', 'NTES', 'OTIS',
       'CTVA', 'CMA', 'CTXS', 'BA', 'ZBH', 'SYK', 'NFLX', 'SIVB', 'OXY',
       'CNC', 'STZ', 'MSI', 'SWK', 'ULTA', 'PEP', 'POOL', 'ICE', 'SCHW',
       'JPM', 'PPL', 'AVY', 'GOOG', 'HES', 'GILD', 'LULU', 'EL', 'TFX',
       'UAA', 'HBI', 'JBHT', 'LOW', 'FLS', 'GIS', 'PTON', 'EFX', 'JKHY',
       'FMC', 'TCOM', 'CME', 'COF', 'CMG', 'AMP', 'PEG', 'HAL', 'VTR',
       'PRGO', 'VFC', 'GRMN', 'MS', 'BAC', 'XEL', 'UNP', 'CARR', 'CHKP',
       'STT', 'BLL', 'WMT', 'TXT', 'WM', 'AVGO', '

In [ ]:
my_columns = ["Ticker", "Stock Price", "Market Cap", "52 Week Change", "Number of Shares to Buy"]
dataframe = pd.DataFrame(columns=my_columns)
for sym in sp500symbols[0:4]:
    if(sym.find(".") > -1):
        sym = sym.replace(".", "-")
    basic = basic_stats(sym)
    complexStats = extended_stock_stats(sym)
    marketCap = parse_value(basic["Market Cap"])
    previousClose = parse_value(basic["Previous Close"])
    fiftyTwoWeekChange = previousClose / (1+parse_value(complexStats["52-Week Change 3"]))
    dataframe = dataframe.append(pd.Series([sym,previousClose,round(marketCap),fiftyTwoWeekChange,"N/A"], index=my_columns), ignore_index=True)
    print(sym, round(marketCap), previousClose)
dataframe.to_csv("lastDataframe_bak.csv")
dataframe

In [ ]:
dataframe.to_csv("lastDataframe.csv")

In [ ]:
portfolio_size = input("How large is your portfolio?")

In [ ]:
portfolio_size = float(portfolio_size)
portfolio_size

In [ ]:
position_size = portfolio_size / len(dataframe.index)
print(position_size)
for i in range(0,len(dataframe.index)):
    dataframe.loc[i,"Number of Shares to Buy"] = math.floor(position_size/dataframe.loc[i,"Stock Price"])

dataframe.to_csv("lastDataframeWithSharesToBuy.csv")
dataframe

In [6]:
metadataAndPrice = metadata_price_action(symbol, timeInterval, timeRange)
metadata = metadataAndPrice["metadata"]
metadata

{'currency': 'USD',
 'symbol': 'TTWO',
 'exchangeName': 'NMS',
 'instrumentType': 'EQUITY',
 'firstTradeDate': 861111000,
 'regularMarketTime': 1614373202,
 'gmtoffset': -18000,
 'timezone': 'EST',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 184.46,
 'chartPreviousClose': 109.47,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EST',
   'start': 1614330000,
   'end': 1614349800,
   'gmtoffset': -18000},
  'regular': {'timezone': 'EST',
   'start': 1614349800,
   'end': 1614373200,
   'gmtoffset': -18000},
  'post': {'timezone': 'EST',
   'start': 1614373200,
   'end': 1614387600,
   'gmtoffset': -18000}},
 'dataGranularity': '1d',
 'range': '1y',
 'validRanges': ['1d',
  '5d',
  '1mo',
  '3mo',
  '6mo',
  '1y',
  '2y',
  '5y',
  '10y',
  'ytd',
  'max']}

In [7]:
price = metadataAndPrice["price"]
price

,timestamp,high,open,close,low,volume
0,1582813800,111.959999,107.559998,107.839996,106.220001,2502800
1,1582900200,109.279999,106.559998,107.480003,105.269997,3385300
2,1583159400,112.129997,108.580002,112.059998,107.919998,2321900
3,1583245800,115.040001,111.750000,111.220001,109.809998,2073500
4,1583332200,119.790001,113.440002,119.419998,113.440002,2573300
...,...,...,...,...,...,...
248,1614004200,195.399994,191.619995,189.850006,189.520004,1704700
249,1614090600,187.139999,184.050003,185.949997,181.110001,2062400
250,1614177000,187.429993,184.800003,186.990005,182.380005,1218900
251,1614263400,188.500000,186.389999,184.679993,181.929993,1217400


In [ ]:
basicStats = basic_stats(symbol)
basicStats

In [ ]:
extStats = extended_stock_stats(symbol)
extStats

In [9]:
price.to_csv("ttwo.csv")